# Introdução ao Django: Python 

[Aprenda Python com Jupyter](https://github.com/jeanto/python_django_course_notebook) by [Jean Nunes](https://jeanto.github.io/jeannunes)   
Code license: [GNU-GPL v3](https://www.gnu.org/licenses/gpl-3.0.en.html)

---

![Arquitetura MTV do Django](mvt.png)

# **1. Principais Arquivos e Diretórios de um Projeto Django**

Ao criar um projeto Django, a estrutura de diretórios e arquivos é organizada para facilitar o desenvolvimento e a manutenção. Veja os principais componentes:

## Estrutura Básica

```
meuprojeto/
│
├── manage.py
├── meuprojeto/
│   ├── __init__.py
│   ├── settings.py
│   ├── urls.py
│   ├── asgi.py
│   └── wsgi.py
└── meuapp/
    ├── __init__.py
    ├── admin.py
    ├── apps.py
    ├── migrations/
    │   └── __init__.py
    ├── models.py
    ├── tests.py
    ├── views.py
    └── templates/
```

### Descrição dos principais arquivos e pastas:

- **manage.py**: Script utilitário para comandos administrativos (rodar servidor, migrações, criar superusuário, etc).
- **meuprojeto/**: Diretório do projeto, contém configurações globais.
  - **settings.py**: Configurações do projeto (banco de dados, apps instalados, etc).
  - **urls.py**: Arquivo principal de rotas do projeto.
  - **asgi.py / wsgi.py**: Pontos de entrada para servidores ASGI/WSGI.
- **meuapp/**: Diretório de um app Django (você pode ter vários apps).
  - **models.py**: Definição dos modelos (tabelas do banco).
  - **views.py**: Definição das views (lógica de resposta às requisições).
  - **admin.py**: Registro dos modelos no admin do Django.
  - **apps.py**: Configurações do app.
  - **migrations/**: Arquivos de migração do banco de dados.
  - **templates/**: Templates HTML do app.




### Conceito de Rotas (URLs) no Django

No Django, as **rotas** (ou URLs) determinam qual view será chamada para cada endereço acessado pelo usuário. O roteamento é feito nos arquivos `urls.py`.

#### Exemplo de urls.py do projeto

```python
# meuprojeto/urls.py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('meuapp.urls')),  # Inclui as rotas do app
]
```

#### Exemplo de urls.py do app

```python
# meuapp/urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('', views.index, name='index'),
    path('doadores/', views.listar_doadores, name='listar_doadores'),
    path('doadores/<int:id>/', views.detalhe_doador, name='detalhe_doador'),
]
```

- `''` → Chama a view `index` para a página inicial.
- `'doadores/'` → Chama a view `listar_doadores` para listar todos os doadores.
- `'doadores/<int:id>/'` → Chama a view `detalhe_doador` para mostrar detalhes de um doador específico.

#### Como funciona o fluxo de rotas

1. O usuário acessa uma URL.
2. O Django procura a correspondência no `urls.py` do projeto.
3. Se encontrar um `include`, procura no `urls.py` do app correspondente.
4. Quando encontra a rota, executa a view associada.


#### Resumindo

- **Arquivos e diretórios**: Organizam o código e as configurações do projeto.
- **Rotas (URLs)**: Definem qual view será executada para cada endereço acessado.
- **Views**: Recebem as requisições, processam dados e retornam respostas (geralmente templates renderizados).

Essa estrutura torna o desenvolvimento com Django organizado, escalável e fácil de manter.

---

# **2. Arquitetura MTV do Django**

O Django utiliza o padrão de arquitetura **MTV**, que significa **Model-Template-View**. Esse padrão é semelhante ao conhecido MVC (Model-View-Controller), mas com algumas adaptações para o contexto do framework. Veja como cada componente funciona:


## Model (Modelo)

O **Model** representa a estrutura dos dados da aplicação. Ele define as tabelas do banco de dados, seus campos e as regras de negócio associadas. Cada modelo é uma classe Python que herda de `django.db.models.Model`.

**Exemplo:**
```python
from django.db import models

class Doador(models.Model):
    nome = models.CharField(max_length=100)
    cpf = models.CharField(max_length=11)
    # outros campos...


## Template (Template)

O **Template** é responsável pela camada de apresentação, ou seja, pelo que o usuário vê na interface. Os templates são arquivos HTML (podendo conter tags especiais do Django) que exibem os dados enviados pelas views.

**Exemplo de template (`listar_doadores.html`):**
```html
<h1>Lista de Doadores</h1>
<ul>
  {% for doador in page_obj %}
    <li>{{ doador.nome }} - {{ doador.cpf }}</li>
  {% endfor %}
</ul>
```


## View (Visão)

A **View** é responsável por receber as requisições do usuário, processar os dados (consultando os modelos, por exemplo) e retornar uma resposta (geralmente um template renderizado). No Django, as views são funções ou classes Python.

**Exemplo:**
```python
from django.shortcuts import render
from .models import Doador

def listar_doadores(request):
    doadores = Doador.objects.all().order_by('nome')
    return render(request, 'listar_doadores.html', {'page_obj': doadores})
```




### Como funciona o fluxo MTV?

1. **Usuário faz uma requisição** (ex: acessa uma URL).
2. **View** recebe a requisição, processa os dados (usando os Models) e seleciona o Template adequado.
3. **Template** é renderizado com os dados fornecidos pela View.
4. **Resposta** é enviada ao navegador do usuário.



#### Resumindo

- **Model:** Gerencia os dados e regras de negócio.
- **Template:** Define a apresentação dos dados.
- **View:** Faz a ligação entre Model e Template, processando as requisições.

Essa separação facilita a manutenção, reutilização e organização do código em projetos Django.

---

# **3. Criando novo _app_**

Para criar um _app_ administrativo chamado `sndot_admin` com uma tela de login (`acesso_restrito.html`) e configurá-lo para ser acessado a partir do `index.html`, vamos seguir o passo a passo para criar um `app` no django.


## 1. Criar o App Django `sndot_admin`

Abra seu terminal na raiz do projeto Django (onde está o manage.py) e execute o seguinte comando:

```shell
python manage.py startapp sndot_admin
```

Este comando criará uma nova pasta `sndot_admin` com a estrutura básica de um _app_ Django.

## 2. Registrar o _App_ no Projeto

Você precisa informar ao Django que o novo _app_ `sndot_admin` existe. Abra o arquivo setup/settings.py e adicione `'sndot_admin.apps.SndotAdminConfig'` à lista **INSTALLED_APPS**:

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'sndot.apps.SndotConfig',
    'sndot_admin.apps.SndotAdminConfig',
]
```

## 3. Criar o Template login.html

Dentro da pasta `sndot_admin`, crie uma nova pasta chamada templates, e dentro dela crie o arquivo _login.html_.

Este arquivo conterá a estrutura da sua tela de login.

```html
{% extends 'base.html' %}

{% block title %}Administrador do Sistema{% endblock %}

{% block content %}
<div class="min-h-screen flex items-center justify-center bg-gray-100 p-4">
    <div class="bg-white p-8 rounded-lg shadow-md w-full max-w-md">
        <h2 class="text-2xl font-bold text-center text-green-600 mb-6">Login</h2>
        {% if messages %}
            <ul class="messages mb-4">
                {% for message in messages %}
                    <li class="{{ message.tags }} p-2 mb-2 rounded-md {% if 'error' in message.tags %}bg-red-100 text-red-800{% elif 'success' in message.tags %}bg-green-100 text-green-800{% endif %}">
                        {{ message }}
                    </li>
                {% endfor %}
            </ul>
        {% endif %}
        <form method="post" class="space-y-4">
            {% csrf_token %}
            <div>
                <label for="username" class="block text-sm font-medium text-gray-700">Nome de Usuário:</label>
                <input type="text" id="username" name="username" required
                       class="mt-1 block w-full px-3 py-2 border border-gray-300 rounded-md shadow-sm focus:outline-none focus:ring-green-500 focus:border-green-500 sm:text-sm">
            </div>
            <div>
                <label for="password" class="block text-sm font-medium text-gray-700">Senha:</label>
                <input type="password" id="password" name="password" required
                       class="mt-1 block w-full px-3 py-2 border border-gray-300 rounded-md shadow-sm focus:outline-none focus:ring-green-500 focus:border-green-500 sm:text-sm">
            </div>
            <div>
                <button type="submit"
                        class="w-full flex justify-center py-2 px-4 border border-transparent rounded-md shadow-sm text-sm font-medium text-white bg-green-600 hover:bg-green-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-green-500">
                    Entrar
                </button>
            </div>
        </form>
        <p class="mt-6 text-center text-sm text-gray-600">
            Esqueceu sua senha? <a href="#" class="font-medium text-green-600 hover:text-green-500">Recuperar Senha</a>
        </p>
    </div>
</div>
{% endblock %}

```

## 4. Criar a _View_ para o _App_ `sndot_admin`

Abra o arquivo `sndot_admin/views.py` e adicione as funções que renderizarão o template de *login* e *logout*.

```python
def painel_admin(request):
    """
    View para a página administrativa.
    Aqui você pode adicionar lógica para exibir dados administrativos,
    como estatísticas, gráficos, etc.
    """
    # Exemplo de lógica para exibir dados administrativos
    # Você pode substituir isso por qualquer lógica que desejar
    context = {
        'titulo': 'Painel Administrativo',
        'mensagem': 'Bem-vindo ao painel administrativo!'
    }
    return render(request, 'admin.html', context)

def logar(request):
    """
    View para a tela de login administrativo.
    Lida com a exibição do formulário de login (GET)
    e com a autenticação do usuário (POST).
    """
    if request.method == 'POST':
        username = request.POST.get('username')
        password = request.POST.get('password')

        # Tenta autenticar o usuário
        user = authenticate(request, username=username, password=password)

        if user is not None:
            # Se o usuário for autenticado, faça o login
            login(request, user)
            messages.success(request, f'Bem-vindo, {user.username}!')
            return redirect('sndot_admin:painel_admin')
        else:
            # Se a autenticação falhar
            messages.error(request, 'Nome de usuário ou senha inválidos.')
    
    # Renderiza a página de login para requisições GET ou falha de POST
    return render(request, 'login.html')

def logout(request):
    """
    View para realizar o logout do usuário administrativo.
    """
    logout(request)
    messages.info(request, 'Você foi desconectado.')
    return redirect('sndot_admin:login') # Redireciona de volta para a tela de login
```

## 5. Criar o Arquivo `urls.py` para o _App_ `sndot_admin`

Dentro da pasta `sndot_admin`, crie um novo arquivo chamado `urls.py`. Este arquivo definirá as URLs específicas para o _app_ administrativo.

```python
from django.urls import path
from . import views

app_name = 'sndot_admin' # Define o namespace do app

urlpatterns = [
    path('login/', views.logar, name='login'),
    path('logout/', views.logout, name='logout'),
    path('painel-admin/', views.painel_admin, name='painel_admin'),
]
```

## 6. Incluir as URLs do _App_ `sndot_admin` no urls.py Principal do Projeto

Agora, você precisa incluir as URLs do seu novo app no arquivo `urls.py` principal do seu projeto (geralmente setup/urls.py).

```python
urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('sndot.urls')),
    path('sndot-admin/', include('sndot_admin.urls', namespace='sndot_admin')),
]
```

## 7. Atualizar o _base.html_ para Redirecionar

Finalmente, você precisa adicionar um link na sua página _base.html_ que aponte para a nova tela de login administrativo.

```html
<a href="{% url 'sndot_admin:login' %}" class="text-white text-sm hover:text-green-200 transition-colors duration-300">Acesso Restrito</a>
```

- {% url 'sndot_admin:login' %}: Esta é a _tag_ de _template_ do Django que gera a URL.

    - `'sndot_admin'` é o _app_name_ que definimos no sndot_admin/urls.py
    - `'login'` é o _name_ que demos ao _path_ da _view_ de login dentro de sndot_admin/urls.py.